# CDCS Data Management

This Notebook details the functions and interactions for managing data in a CDCS instance.  Primarily, this means having an account on the database allowing you to log in to create and modify records. 

Notes on CDCS design:

- Each record is assigned to a user and can be assigned to a workspace.
- Any records without a workspace are only accessible to the assigned user.
- A workspace serves as a group of records that can be made accessible to specific users.
- Each CDCS instance has a "Global Public Workspace" that is meant for all users to be able to see.

__CAUTION!!! Do not run all cells in this Notebook at once!  Some operations (in 6. Managing templates) will permanently add testing content to your CDCS database that cannot be removed without resetting your database!  Those operations are marked below and should only be used when needed!__

In [1]:
from pathlib import Path

import cdcs
from cdcs import CDCS

print('Notebook executed for cdcs version', cdcs.__version__)

Notebook executed for cdcs version 0.2.3


## 1. Class initialization

A CDCS client manager can be initialized by passing it the host url and authenication information.

Parameters

- __host__: (*str*) URL for the database's server.
- __username__: (*str, optional*) Username of desired account on the server. A prompt will ask for the username if not given.
- __password__: (*str, optional*) Password of desired account on the server.  A prompt will ask for the password if not given.
- __auth__: (*tuple, optional*) Auth tuple to enable Basic/Digest/Custom HTTP Auth.  Alternative to giving username and password seperately.
- __cert__: (*str, optional*) if String, path to ssl client cert file (.pem). If Tuple, (‘cert’, ‘key’) pair.
- __certification__: (*str, optional*) Alias for cert. Retained for compatibility.
- __verify__: (*bool or str, optional*) Either a boolean, in which case it controls whether we verify the server’s TLS certificate, or a string, in which case it must be a path to a CA bundle to use. Defaults to True.
- __cdcsversion__: (*str, optional*) For CDCS versions 2.X.X, this allows for specifying the full CDCS version to ensure the class methods perform the correct REST calls.  This can be specified as "#.#.#", or if None is given will default to "2.15.0".  For CDCS versions 3.X.X, this is ignored as version info is obtained directly from the database.

In [3]:
curator = CDCS('https://potentials.nist.gov/', username='lmh1')
print(curator.cdcsversion)

Enter password for lmh1 @ https://potentials.nist.gov: ········


(3, 6, 0)


## 2. Query data

The query() method will return *all* matching records that you have access to.

Parameters

- __template__: (*list, str, pandas.Series or pandas.DataFrame, optional*) One or more templates or template titles to limit the search by.
- __title__: (*str, optional*) Record title to limit the search by.
- __keyword__: (*str or list, optional*) Keyword(s) to use for a string-based search of record content.  Only records containing all keywords will be returned. 
- __mongoquery__: (*str or dict, optional*) Mongodb find query to use in limiting searches by record element fields.  Note: only record parsing is supported, not field projection.
- __page__: (*int or None, optional*) If an int, then will return results only for that page of 10 records.  If None (default), then results for all pages will be compiled and returned.
- __parse_dates__: (*bool, optional*) If True (default) then date fields will automatically be parsed into pandas.Timestamp objects.  If False they will be left as str values.
- __progress_bar__: (*bool, optional*) If True (default) a progress bar will be displayed for multi-page query results.

Returns

- (*pandas.DataFrame*) All records matching the search request

Specify a template in the database to interact with

In [3]:
# Note: template should the name of a template in the database you are accessing!
template = 'FAQ'

Use query to fetch records

In [4]:
records = curator.query(template=template)
records

,id,template,workspace,user_id,title,creation_date,last_modification_date,last_change_date,xml_content,template_title
0,620851,6,1,2,graphs,2021-08-26 13:15:59.839000-04:00,2021-08-26 13:15:59.839000-04:00,2021-08-26 13:16:00.155000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
1,620853,6,1,2,lammps,2021-08-26 13:15:59.365000-04:00,2021-08-26 13:15:59.365000-04:00,2021-08-26 13:15:59.674000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
2,620856,6,1,2,manuscript,2021-08-26 13:15:58.878000-04:00,2021-08-26 13:15:58.878000-04:00,2021-08-26 13:15:59.197000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
3,620855,6,1,2,ref,2021-08-26 13:15:58.407000-04:00,2021-08-26 13:15:58.407000-04:00,2021-08-26 13:15:58.715000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
4,620854,6,1,2,formats,2021-08-26 13:15:57.919000-04:00,2021-08-26 13:15:57.919000-04:00,2021-08-26 13:15:58.236000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
5,620852,6,1,2,faq,2021-08-26 13:15:57.438000-04:00,2021-08-26 13:15:57.438000-04:00,2021-08-26 13:15:57.750000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ
6,620857,6,1,2,submit,2021-08-26 13:15:56.939000-04:00,2021-08-26 13:15:56.939000-04:00,2021-08-26 13:15:57.264000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",FAQ


Pick the first record, and see its xml contents

In [6]:
record = records.iloc[0]
content = record.xml_content
print(content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I've downloaded a file and plotted it, but the graphs don't look like the figures in the paper.  Why?</question><answer><![CDATA[There can be several reasons for this.  One is file formatting.   Different developers and software packages use different data and file formats to present their interatomic potentials.</p>  <p>Even with the same file/data format, invariant transformations in the EAM format mean that different parameterizations of the interatomic potentials can look completely different but yield the same physical properties.  This is especially true for alloys.  The issue of invariant transformations is discussed in several places.  Among them are: </p> <UL> <LI>Y. Mishin, "Interatomic potentials for metals," in <em>Handbook of Materials Modeling</em>, edited by S. Yip (Springer, Dordrect, The Netherlands, 2005), Chap. 2.2, pp. 459-478. <LI>A.E. Carlsson, "Beyond pair potentials in elemental transition me

## 3. Manage records

The methods associated with managing records requires having an account with the corresponding database and the correct privileges.

### 3.1. Upload a new record

New records can be uploaded to the database using the upload_record() method.

Parameters

- **template** (*str or pandas.Series*) The template or template title to associate with the record.
- **filename** (*str, optional*) Name of an XML file whose contents are to be uploaded.  Either filename or content required.
- **content** (*str or bytes, optional*) String content to upload. Either filename or content required.
- **title** (*str, optional*) Title to save the record as.  Optional if filename is given (title will be taken as filename without ext).
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being uploaded.
- **duplicatecheck** (*bool, optional*) If True (default), then a ValueError will be raised if a record already exists with the same template and title.  If False, no check is performed possibly allowing for multiple records with the same title to exist in the database.  Note: this check only searches the records that you have access to, so duplicates are possible from other users.
- **auto_set_pid_off** (*bool, optional*) If True the auto_set PID will automatically be turned off before and  turned on after uploading. Not  needed if the record has no PID valu   or pid_xpath set.  Convenient if a single record with a P    value is being uploaded.  For uploading multiple records with PID val     use the auto_set_pid_off context manager around batch uploads      manually turn the setting on/off with auto_set_pid.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Use the content from the record queried above to upload a "test" record.

In [7]:
title = 'testrecord1'
curator.upload_record(template=template, title=title, content=content, verbose=True)

record testrecord1 (627945) successfully uploaded.


Alternatively, the names of any local XML files can be specified and the method will automatically read the contents.  The file name without path and extension will be used for the title if no title is given.

Also, note that the workspace is being set to 'Global Public Workspace'.

In [8]:
# Save content to local file
filename = 'testrecord2.xml'
with open(filename, 'w') as f:
    f.write(content)
    
# Upload from file
curator.upload_record(template=template, filename=filename, verbose=True,
                      workspace='Global Public Workspace')

# Delete local file (keep working directory clean)
Path(filename).unlink()

record testrecord2 (627946) successfully uploaded.
record 627946 assigned to workspace 1


### 3.2. Retrieve data records

The records that you own can be accessed using the get_records() and get_record() methods. get_records() will fetch all matching records, while get_record() will fetch a single record if exactly one match is found and will throw an error otherwise.

__NOTE__: The behavior of get_records() is different for CDCS versions 2.X.X and 3.X.X.  For versions 2.X.X, get_records will attempt to return all matching records at once.  For versions 3.X.X, get_records behaves similiarly to query in that matching records are returned in page batches of ten.

Parameters

- __template__ (*str or pandas.Series, optional*) The template or template title to limit the search by.
- __title__ (*str, optional*) The data record title to limit the search by.
- __page__ (*int or None, optional*) If an int, then will return results only for that page of 10 records. If None (default), then results for all pages will be compiled and returned.  Only used for CDCS versions 3.X.X. 
- __parse_dates__ (*bool, optional*) If True (default) then date fields will automatically be parsed into pandas.Timestamp objects.  If False they will be left as str values.
- __progress_bar__ (*bool, optional*) If True (default) a progress bar will be displayed for multi-page query results. Only used for CDCS versions 3.X.X.


In [9]:
curator.get_records(template=template)

,id,template,workspace,user_id,title,checksum,creation_date,last_modification_date,last_change_date,xml_content
0,627946,6,1.0,2,testrecord2,None,2024-03-25 14:30:27.389053-04:00,2024-03-25 14:30:27.389053-04:00,2024-03-25 14:30:28.282758-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
1,627945,6,NaN,2,testrecord1,None,2024-03-25 14:30:24.044186-04:00,2024-03-25 14:30:24.044186-04:00,2024-03-25 14:30:24.044186-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
2,620851,6,1.0,2,graphs,None,2021-08-26 09:15:59.839000-04:00,2021-08-26 09:15:59.839000-04:00,2021-08-26 09:16:00.155000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
3,620853,6,1.0,2,lammps,None,2021-08-26 09:15:59.365000-04:00,2021-08-26 09:15:59.365000-04:00,2021-08-26 09:15:59.674000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
4,620856,6,1.0,2,manuscript,None,2021-08-26 09:15:58.878000-04:00,2021-08-26 09:15:58.878000-04:00,2021-08-26 09:15:59.197000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
5,620855,6,1.0,2,ref,None,2021-08-26 09:15:58.407000-04:00,2021-08-26 09:15:58.407000-04:00,2021-08-26 09:15:58.715000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
6,620854,6,1.0,2,formats,None,2021-08-26 09:15:57.919000-04:00,2021-08-26 09:15:57.919000-04:00,2021-08-26 09:15:58.236000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
7,620852,6,1.0,2,faq,None,2021-08-26 09:15:57.438000-04:00,2021-08-26 09:15:57.438000-04:00,2021-08-26 09:15:57.750000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."
8,620857,6,1.0,2,submit,None,2021-08-26 09:15:56.939000-04:00,2021-08-26 09:15:56.939000-04:00,2021-08-26 09:15:57.264000-04:00,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch..."


In [10]:
record = curator.get_record(template=template, title='testrecord2', parse_dates=False)
print(record)

id                                                                   627946
template                                                                  6
workspace                                                                 1
user_id                                                                   2
title                                                           testrecord2
checksum                                                               None
creation_date                              2024-03-25T14:30:27.389053-04:00
last_modification_date                     2024-03-25T14:30:27.389053-04:00
last_change_date                           2024-03-25T14:30:28.282758-04:00
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
Name: 0, dtype: object


In [11]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I've downloaded a file and plotted it, but the graphs don't look like the figures in the paper.  Why?</question><answer><![CDATA[There can be several reasons for this.  One is file formatting.   Different developers and software packages use different data and file formats to present their interatomic potentials.</p>  <p>Even with the same file/data format, invariant transformations in the EAM format mean that different parameterizations of the interatomic potentials can look completely different but yield the same physical properties.  This is especially true for alloys.  The issue of invariant transformations is discussed in several places.  Among them are: </p> <UL> <LI>Y. Mishin, "Interatomic potentials for metals," in <em>Handbook of Materials Modeling</em>, edited by S. Yip (Springer, Dordrect, The Netherlands, 2005), Chap. 2.2, pp. 459-478. <LI>A.E. Carlsson, "Beyond pair potentials in elemental transition me

### 3.3. Update an existing record

The content of a record in the database can be changed using the update_record() method.  Note that this only changes the record's content and all other metadata (user, workspace, database id,...) will remain unchanged.

- **record** (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the template and title parameters are ignored if given. Can contain the new conte
- **template** (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- **title** (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- **filename** (*str or Path, optional*) Path to file containing the new record content to upload. Either filename or content required.
- **content** (*str or bytes, optional*) New content to upload. Either filename or content required.
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being updated.
- **auto_set_pid_off** (*bool, optional*) If True the auto_set PID will automatically be turned off before and turned on after uploading. Not  needed if the record has no PID value or pid_xpath set.  Convenient if a single record with a PID value is being uploaded.  For uploading multiple records with PID values use the auto_set_pid_off context manager around batch uploads, or manually turn the setting on/off with auto_set_pid.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Change the content of testrecord2 by replacing "download" with "find"

In [12]:
new_content = record.xml_content.replace('download', 'find')
print(new_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I've finded a file and plotted it, but the graphs don't look like the figures in the paper.  Why?</question><answer><![CDATA[There can be several reasons for this.  One is file formatting.   Different developers and software packages use different data and file formats to present their interatomic potentials.</p>  <p>Even with the same file/data format, invariant transformations in the EAM format mean that different parameterizations of the interatomic potentials can look completely different but yield the same physical properties.  This is especially true for alloys.  The issue of invariant transformations is discussed in several places.  Among them are: </p> <UL> <LI>Y. Mishin, "Interatomic potentials for metals," in <em>Handbook of Materials Modeling</em>, edited by S. Yip (Springer, Dordrect, The Netherlands, 2005), Chap. 2.2, pp. 459-478. <LI>A.E. Carlsson, "Beyond pair potentials in elemental transition metals

In [13]:
curator.update_record(record=record, content=new_content, verbose=True)

record testrecord2 (627946) has been updated.


Retrieve the record again from the database showing nothing has changed except modification date and content

In [14]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                                   627946
template                                                                  6
workspace                                                                 1
user_id                                                                   2
title                                                           testrecord2
checksum                                                               None
creation_date                              2024-03-25 14:30:27.389053-04:00
last_modification_date                     2024-03-25 14:30:37.305797-04:00
last_change_date                           2024-03-25 14:30:37.314193-04:00
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
Name: 0, dtype: object


In [15]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>I've finded a file and plotted it, but the graphs don't look like the figures in the paper.  Why?</question><answer><![CDATA[There can be several reasons for this.  One is file formatting.   Different developers and software packages use different data and file formats to present their interatomic potentials.</p>  <p>Even with the same file/data format, invariant transformations in the EAM format mean that different parameterizations of the interatomic potentials can look completely different but yield the same physical properties.  This is especially true for alloys.  The issue of invariant transformations is discussed in several places.  Among them are: </p> <UL> <LI>Y. Mishin, "Interatomic potentials for metals," in <em>Handbook of Materials Modeling</em>, edited by S. Yip (Springer, Dordrect, The Netherlands, 2005), Chap. 2.2, pp. 459-478. <LI>A.E. Carlsson, "Beyond pair potentials in elemental transition metals

### 3.4. Assign records to a workspace

The upload_record() and update_record() methods give the option to assign each record to a workspace if wanted.  After a record is added, the workspace can also be changed using the assign_records() method.

Parameters

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the records to.
- __records__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected records to assign to the workspace.  Cannot be given with ids, template, or title.
- __ids__ (*str or list, optional*) The ID(s) of the records to assign to the workspace.  Selecting records using ids has the least overhead. Cannot be given with records, template, or title.
- __template__ (*str or pandas.Series, optional*) The template or template title of records to assign to the workspace.  Cannot be given with records or ids.
- __title__ (*str, optional*) The title of a record to assign to the workspace. Cannot be given with records or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [16]:
curator.assign_records(workspace='Global Public Workspace', title='testrecord1', template=template, verbose=True)

record 627945 assigned to workspace 1


### 3.5. Delete a record

A record can also be deleted by the assigned user using the delete_record() method.

Parameters

- __record__ (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the other parameters are ignored.
- __template__ (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- __title__ (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [17]:
# Delete by identifying with title + template
curator.delete_record(title='testrecord1', template=template, verbose=True)

record testrecord1 (627945) has been deleted.


In [18]:
# Get a record first
record = curator.get_record(title='testrecord2', template=template)

# Delete by passing Series
curator.delete_record(record, verbose=True)

record testrecord2 (627946) has been deleted.


## 4. Manage blobs (raw files)

The database can also store any non-record files as blob files that can later be retrieved and downloaded all at once.

Create blob file for testing

In [19]:
filename = 'test_blob.txt'

with open(filename, 'w') as f:
    f.write('This is my blob for testing')

### 4.1. Upload blob

Blobs can be added to the database using the upload_blob() method.  When a record is uploaded, it will be associated with the given filename (minus the path).

Parameters

- __filename__ (*str or Path*) The path/name of the file to upload.
- __blobbytes__ (*bytesIO, optional*) Pre-loaded file contents.  Allows for the contents of open file-like objects to be passed in and associated with the filename.  If not given, then the file filename will be read instead.
- __workspace__ (*str or pandas.Series, optional*) If given, the blob will be assigned to this workspace after successfully being uploaded.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Returns

- __handle__ (*str*) The URL that can be used to retrieve the blob's contents.

In [20]:
handle = curator.upload_blob(filename=filename, verbose=True)
print(handle)

File "test_blob.txt" uploaded as blob "test_blob.txt" (1)
https://potentials.nist.gov/rest/blob/download/1/


### 4.2 Find blob metadata

The list of all blobs currently in the database, or all with an associated filename can be explored with the get_blobs() method.

Parameters

- __filename__ (*str, optional*) The name of the file to limit the search by.

A single blob can also be retrieved using the get_blob() method based on a filename, if it is unique in the database, or a database id.

Parameters

- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with filename
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with id.

__NOTE__: Exploring blob metadata appears to behave differently for some versions of CDCS, with the filename searches only showing blobs that the user owns, while id searches shows any blobs available to the user, either by being owned or in an available workspace.

- Identifying by filename only works for user blobs
- Identifying by id works for all blobs available to you (user + allowed workspaces)

In [21]:
curator.get_blobs()

,id,user_id,filename,handle,blob,workspace,checksum,upload_date,pid
0,1,2,test_blob.txt,https://potentials.nist.gov/rest/blob/download/1/,https://potentials.nist.gov/rest/blob/user_2/t...,None,None,2024-03-25 18:30:52.545142+00:00,https://potentials.nist.gov/pid/rest/local/pot...


In [22]:
blobdata = curator.get_blob(filename=filename)
print(blobdata)

id                                                             1
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download/1/
blob           https://potentials.nist.gov/rest/blob/user_2/t...
workspace                                                   None
checksum                                                    None
upload_date                     2024-03-25 18:30:52.545142+00:00
pid            https://potentials.nist.gov/pid/rest/local/pot...
Name: 0, dtype: object


In [23]:
curator.get_blob(id=blobdata.id)

id                                                             1
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download/1/
blob           https://potentials.nist.gov/rest/blob/1/user_2...
workspace                                                   None
checksum                                                    None
upload_date                     2024-03-25 18:30:52.545142+00:00
pid            https://potentials.nist.gov/pid/rest/local/pot...
dtype: object

### 4.3 Retrieve blob contents

The contents of a blob can then be retrieved using the get_blob_contents() method.

Paramters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.

Returns

- __content__ (*bytes*) The blob's bytes contents.

In [24]:
print(curator.get_blob_contents(filename=filename))

b'This is my blob for testing'


Alternatively, the blob can be saved directly to a local file based on its associated filename with the download_blob() method.

Parameters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __savedir__ (*str or Path, optional*) The directory to save the file to.  Default value uses the current working directory.

In [25]:
curator.download_blob(filename=filename)

### 4.4. Assign blobs to a workspace

Just like records, blobs can also be assigned to workspaces.  This can be done during uploading, or it can be done after uploading using the assign_blobs() method.

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the blobs to.
- __blobs__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected blobs to assign to the workspace.  Cannot be given with ids or filename.
- __ids__ (*str or list, optional*) The ID(s) of the blobs to assign to the workspace.  Selecting blobs using ids has the least overhead. Cannot be given with blobs or filename.
- __filename__ (*str, optional*) The name of the blob file to assign to the workspace.  Cannot be given with blobs or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [26]:
curator.assign_blobs(workspace='Global Public Workspace', filename=filename, verbose=True)

blob 1 assigned to workspace 1


### 4.5. Delete blob

Finally, blobs can also be deleted using the delete_blob() method.

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [27]:
curator.delete_blob(filename=filename, verbose=True)

Successfully deleted blob "test_blob.txt" (1)


In [28]:
Path(filename).unlink()

## 5. Managing workspaces

Workspaces are used to manage access to content stored in a CDCS database. Records and blobs can be assigned to a workspace, then access control is set at the workspace level to all associated content.  The workspace assignment typically falls into three categories:

1. Content not assigned to a workspace will only be accessible to the user who created it.
2. Content assigned to the "Global Public Workspace" can be accessible to anyone who has read access to the database.
3. Alternate workspaces can be used to limit access to some content for only specific users.

### 5.1. Finding workspace metadata

Metadata for the available workspaces can be explored using the get_workspaces() and get_workspace() methods.

Parameters

title: (str, optional) The workspace title to limit the search by.

In [29]:
curator.get_workspaces()

,id,title,owner,is_public
0,1,Global Public Workspace,None,True


In [30]:
curator.get_workspace(title='Global Public Workspace')

id                                 1
title        Global Public Workspace
owner                           None
is_public                       True
Name: 0, dtype: object

The attribute global_workspace also retrieves the Global Public Workspace 

In [31]:
workspace = curator.global_workspace
print(workspace)

id                                 1
title        Global Public Workspace
owner                           None
is_public                       True
Name: 0, dtype: object


## 6. Managing templates

Templates are used to categorize the types of records that a database stores and to validate uploaded record contents against a schema.  Templates are managed differently than other types of CDCS content in that they are version controlled; changes to template contents do not overwrite the current contents but rather get uploaded as a new template version.  Separate entries called template managers then specify which template versions exist, are active/disabled, and which version is the current version. 

### 6.1. Finding template metadata

A list of all available templates can be retrieved using the template_titles attribute.

In [32]:
curator.template_titles

['Potential',
 'relaxed_crystal',
 'FAQ',
 'Action',
 'PotentialProperties',
 'Request',
 'free_surface',
 'stacking_fault',
 'point_defect',
 'crystal_prototype',
 'potential_LAMMPS',
 'Citation',
 'calculation_diatom_scan',
 'calculation_isolated_atom',
 'potential_LAMMPS_KIM',
 'calculation_bond_angle_scan',
 'dislocation',
 'calculation_E_vs_r_scan',
 'calculation_relax_box',
 'calculation_relax_dynamic',
 'calculation_relax_static']

Metadata for the available templates can be explored using the get_templates() and get_template() methods.

Parameters

- __title__ (*str, optional*): The template title to limit the search by.
- __is_disabled__ (*bool, optional*): If True, then disabled templates will be returned.  If False (default), then active templates will be returned.
- __current__ (bool, optional*): If True (default), only current template versions will be returned.
- __useronly__ (*bool, optional*): If True, only a user's templates are returned. If False (default), then all global templates are returned.

In [33]:
curator.get_templates()

,id,user,filename,checksum,content,hash,dependencies,title
0,21,None,Potential.xsd,None,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",bfac108952e884965fe5e1517204b8f38d32b89f,[],Potential
1,2,None,crystal.xsd,None,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",e3dd6b141935f0cfbe3bea611df3467d027658a7,[],relaxed_crystal
2,6,None,record-interatomic-potential-faq.xsd,None,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",2dc0be555da041f782a7435c377ea7a20f28d3cd,[],FAQ
3,7,None,record-interatomic-potential-action.xsd,None,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",f1c3de8a64553d81bd10cfc0cb24dd3cc129c710,[],Action
4,8,None,record-per-potential-properties.xsd,None,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",ea7eeed57e0cea9b15dbda4b32c08266201a7e5a,[],PotentialProperties
5,9,None,Request3,None,"<xsd:schema xmlns:xsd=""http://www.w3.org/2001/...",d108bacceb44a2789544ade71018b7f5ceacca1b,[],Request
6,10,None,surface.xsd,None,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",3d898fb080654874a30166b22d7850ff4c5327af,[],free_surface
7,11,None,fault.xsd,None,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",c60975939fff88aaea384494b901e11f366137c1,[],stacking_fault
8,12,None,defect.xsd,None,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",b077b95055d04a914e0d2f69f82fd47be562a5b5,[],point_defect
9,13,None,prototype.xsd,None,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",a711a977750c1ce42bb6dc35e3e71cfcb25d0304,[],crystal_prototype


In [34]:
curator.get_template('Citation')

id                                                             15
user                                                         None
filename                record-interatomic-potential-citation.xsd
checksum                                                     None
content         <?xml version="1.0" encoding="UTF-8" standalon...
hash                     893099854c8ac392bad133ce71ac34f60407ab6f
dependencies                                                   []
title                                                    Citation
Name: 0, dtype: object

The get_template_managers() method fetches metadata associated with the various versions of templates in the database. 

Parameters

- __title__ (*str, optional*) The template title to limit the search by.
- __is_disabled__ (*bool, optional*) If True, then disabled templates will be returned.  If False (default), then active templates will bereturned.
- __useronly__ (*bool, optional*) If True, only a user's templates are returned. If False (default), then all global templates arereturned.

In [35]:
curator.get_template_managers('Citation')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,15,[15],15,[],Citation,None,False,VersionManager.TemplateVersionManager,2023-03-26T22:51:01.495098-04:00,12


### 6.2. Adding and modifying templates

__NOTE__: The upload and update operations for templates differ from other types of CDCS contents due to the template version control system.  Notably, the content associated with an uploaded template cannot be modified or replaced, simply new versions uploaded and managed.

New templates can be added using upload_template(). This will create a template manager for the template title as well as the first version of that template.

Parameters

- __filename__ (*str, optional*) Name of the XSD schema file to upload for the template.  Optional if title is given (filename will be taken as "title".xsd).
- __content__ (*str or bytes, optional*) String contents of an XSD schema file to upload for the template.  Optional if filename is given as a full path to the XSD file.
- __title__ (*str, optional*) Title to save the template as.  Optional if filename is given (title will be taken as filename without ext).
- __useronly__ (*bool, optional*) If True, the template will be associated only with the user. If False (default), it will be made a global template.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [36]:
title = 'pycdcs-test'
content = """<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<xsd:schema xmlns:xsd="http://www.w3.org/2001/XMLSchema" attributeFormDefault="unqualified" elementFormDefault="unqualified">
  <xsd:element name="pycdcs-test" type="xsd:anyType"/>
</xsd:schema>"""

__WARNING!!! The next cell will permanently add a template version and template manager under the title name above to your CDCS database!!!__

In [37]:
curator.upload_template(title=title, content=content, verbose=True)

template pycdcs-test (22) successfully uploaded.


Calling update_template() will create a new version of a template and associate it with an existing template title.  Options allow for some automatic version management of the new/old templates.

Parameters
    
- __filename__ (*str, optional*) Name of the XSD schema file to upload for the template.  Optional if title is given (filename will be taken as "title".xsd).
- __content__ (*str or bytes, optional*) String contents of an XSD schema file to upload for the template.  Optional if filename is given as a full path to the XSD file.
- __title__ (*str, optional*) Title to save the template as.  Optional if filename is given (title will be taken as filename without ext).
- __template_manager__ (*pandas.Series, optional*) Can be given instead of title if the template_manager info has already been retrieved from the database.
- __set_current__ (*bool, optional*) If True (default), will set the uploaded version of the template to be the current active version.
- __disable_old__ (*bool, optional*) If True, any active versions of the template besides the newly created and the current version (if different) will be disabled.  Default value is False.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

__WARNING!!! The next cell will permanently add another template version under the title name above to your CDCS database!!!__

In [38]:
curator.update_template(title=title, content=content, verbose=True)

template pycdcs-test (23) successfully uploaded.
template with id 23 set as current version


### 6.3. Template version management operations

As templates cannot be modified or deleted, there are a number of operations that allow for template versions and managers to be made less accessible.

Individual template versions can be set as disabled or returned to active status using disable_template() and restore_template(), respectively.  Disabling a template version prevents users from adding records to that template version.  For each template title, a single template version is considered the "current" version. The set_current_template() method changes which template version is the current one.  All three of these methods rely on parameters that identify a specific version of a template.

Parameters
    
- __title__ (*str, optional*) The template title.
- __version__ (*int, optional*) The version of the template to make current.  Required unless template_id is given.  Note that version numbers start at 1.
- __template_manager__ (*pandas.Series, optional*) Template manager information for the template.  This can be given instead of title to avoid querying for the template manager.
- __template_id__ (*str, optional*) The database id for the template to set as current.  If given, then no other parameters are allowed (or needed).
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [39]:
# Use get_template_managers to check status of pycdcs-test versions
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",23,[],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


In [40]:
# "version" is the index of versions to set to current (version starts at 1 not 0!)
curator.set_current_template('pycdcs-test', version=1)
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


In [41]:
# "tempate_id" sets the version based on the version id/number
curator.set_current_template(template_id=19)
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


In [45]:
# Disable the old version of pycdcs-test
curator.disable_template('pycdcs-test', version=2)
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[23],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


In [46]:
# Restore the old version of pycdcs-test
curator.restore_template('pycdcs-test', version=2)
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


Additionally, all versions of a given template can be disabled or restored using disable_template_manager() and restore_template_manager(), respectively.  The parameters for these methods are used to identify the template family to disable/restore. 

Parameters
    
- __title__ (*str, optional*) The template title.
- __template_manager__ (*pandas.Series, optional*) Template manager information for the template.  This can be given instead of title to avoid querying for the template manager.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [47]:
curator.disable_template_manager('pycdcs-test')
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[],pycdcs-test,None,True,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


In [48]:
curator.restore_template_manager('pycdcs-test')
curator.get_template_managers('pycdcs-test')

,id,versions,current,disabled_versions,title,user,is_disabled,_cls,creation_date,display_rank
0,22,"[22, 23]",22,[],pycdcs-test,None,False,VersionManager.TemplateVersionManager,2024-03-25T14:31:57.467260-04:00,None


### 6.4. Template migration and validation

Each record is associated with a specific template. Migration allows for records to be associated with a different template instead, as long as the record contents can be validated against the new template's schema.  This type of operation is typically done when a new version of a template is added that extends the current template.  Validation performs the schema validation checks for the records against the new template without migrating the records to the new template.

These operations Will hopefully be added soon as the proper use of the REST calls are being worked out. 

## 7. Managing XSLTs 

XSLT files allow for the XML content of records to be transformed. This is used by CDCS for a variety of purposes
- Transformations to HTML can be used to render the contents of record entries into user-friendly web pages.
- Transformations can also be specified to convert the record contents into other text-based formats that users may wish to download.
- A transformation can also be used for data migration when a new template version is not backwards compatible with the current template version.  Here, the transformation would be XML to XML so that the content is valid with the new template schema. 

### 7.1. Exploring existing XSLTs

Similar to other content types, get_xslts() and get_xslt() fetch all or one matching XSLT files from the database.  Each XSLT can be identified by its name or filename.

Parameters
 
- __name__ (*str, optional*) The xslt name to limit the search by.
- __filename__ (*str, optional*) The xslt filename to limit the search by.

In [49]:
curator.get_xslts()

,id,name,filename,checksum,content
0,1,potential_LAMMPS,potential_LAMMPS.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
1,2,action,action.xsl.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<xsl:s..."
2,3,calculation_isolated_atom,calculation_isolated_atom.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
3,4,stacking_fault,stacking_fault.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
4,5,potential_LAMMPS_KIM,potential_LAMMPS_KIM.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
5,6,free_surface,free_surface.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
6,7,crystal_prototype,crystal_prototype.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
7,8,crystal_prototype_short,crystal_prototype_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
8,9,calculation_E_vs_r_scan,calculation_E_vs_r_scan.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
9,10,relaxed_crystal_short,relaxed_crystal_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."


### 7.2. Uploading a new XSLT

New XSLTs can be uploaded using upload_xslt().

Parameters
    
- __name__ (*str, optional*) The name to associate with the XSLT file.  Optional if filename is given as name will be taken as filename without its extension.
- __filename__ (*str, optional*) The filename to associate with the XSLT file.  Optional if name and content are given.  If not given, filename will be set to name + '.xsl'. Will read the file contents if the file exists and content is not given.
- __content__ (*str or bytes, optional*) XSLT file contents.  Optional if filename is given and points to a file that exists.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.


In [50]:
name = 'test'
content = '<?xml version="1.0" encoding="UTF-8"?><xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform" xmlns="http://www.w3.org/TR/xhtml1/strict"> <xsl:output method="html" encoding="utf-8" indent="yes" /></xsl:stylesheet>'

curator.upload_xslt(name=name, content=content)

curator.get_xslts()

,id,name,filename,checksum,content
0,1,potential_LAMMPS,potential_LAMMPS.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
1,2,action,action.xsl.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<xsl:s..."
2,3,calculation_isolated_atom,calculation_isolated_atom.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
3,4,stacking_fault,stacking_fault.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
4,5,potential_LAMMPS_KIM,potential_LAMMPS_KIM.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
5,6,free_surface,free_surface.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
6,7,crystal_prototype,crystal_prototype.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
7,8,crystal_prototype_short,crystal_prototype_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
8,9,calculation_E_vs_r_scan,calculation_E_vs_r_scan.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
9,10,relaxed_crystal_short,relaxed_crystal_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."


### 7.3. Modifying an existing XSLT

Any uploaded XSLT entries can be modified and updated using update_xslt(). Note that there are additional parameters compared to other update operations as the name and filename fields can be updated in addition to the content.

Parameters
    
- __name__ (*str, optional*) An xslt name. Will be used to identify the existing xslt entry to update if neither xslt nor xslt_id parameters are given. If either xslt or xslt_id are given, this can be used to assign a new name to the entry.
- __filename__ (*str, optional*) An xslt filename. Will be used to identify the existing xslt entry to update if neither xslt nor xslt_id parameters are given. If either xslt or xslt_id are given, this can be used to assign a new filename to the entry.
- __content__ (*str, optional*) New xsl content to assign to the entry.
- __newname__ (*str, optional*) New name to assign to the entry.
- __newfilename__ (*str, optional*) New filename to assign to the entry.
- __xslt__ (*pd.Series, optional*) The xslt entry information for the entry that is being updated.  
- __xslt_id__ (*str, optional*) The database id that uniquely identifies the xslt entry. 
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.


In [51]:
name = 'test'
content = '<?xml version="1.0" encoding="UTF-8"?><xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform" xmlns="http://www.w3.org/TR/xhtml1/strict"> <xsl:output method="html" encoding="utf-8" indent="yes" /></xsl:stylesheet>'

curator.update_xslt(name=name, content=content)

curator.get_xslts()

,id,name,filename,checksum,content
0,1,potential_LAMMPS,potential_LAMMPS.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
1,2,action,action.xsl.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<xsl:s..."
2,3,calculation_isolated_atom,calculation_isolated_atom.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
3,4,stacking_fault,stacking_fault.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
4,5,potential_LAMMPS_KIM,potential_LAMMPS_KIM.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
5,6,free_surface,free_surface.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
6,7,crystal_prototype,crystal_prototype.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
7,8,crystal_prototype_short,crystal_prototype_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
8,9,calculation_E_vs_r_scan,calculation_E_vs_r_scan.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
9,10,relaxed_crystal_short,relaxed_crystal_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."


### 7.4. Deleting an existing XSLT

Any XSLT files can also be removed from the database using delete_xslt().

Parameters
    
- __name__ (*str, optional*) Values for name and filename can be specified to try to uniquely identify the XSLT entry to delete.  Cannot be combined with xslt or xslt_id as they uniquely identify the XSLT on their own.
- __filename__ (*str, optional*) Values for name and filename can be specified to try to uniquely identify the XSLT entry to delete.  Cannot be combined with xslt or xslt_id as they uniquely identify the XSLT on their own.
- __xslt__ (*pd.Series, optional*) The xslt entry information for the entry that is to be deleted.  Cannot be combined with name, filename or xslt_id.
- __xslt_id__ (* str, optional*) The database id that uniquely identifies the xslt entry to delete.  Cannot be combined with name, filename or xslt.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [52]:
curator.delete_xslt(name=name)

curator.get_xslts()

,id,name,filename,checksum,content
0,1,potential_LAMMPS,potential_LAMMPS.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
1,2,action,action.xsl.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<xsl:s..."
2,3,calculation_isolated_atom,calculation_isolated_atom.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
3,4,stacking_fault,stacking_fault.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
4,5,potential_LAMMPS_KIM,potential_LAMMPS_KIM.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
5,6,free_surface,free_surface.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
6,7,crystal_prototype,crystal_prototype.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
7,8,crystal_prototype_short,crystal_prototype_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
8,9,calculation_E_vs_r_scan,calculation_E_vs_r_scan.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."
9,10,relaxed_crystal_short,relaxed_crystal_short.xsl,None,"<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<xsl..."


## 8. Managing PID XPATHs

To help make data in a CDCS more compliant with FAIR practices, a PID (persistent identifier) can be assigned to each record entry and blob.  Without a PID, the URL for accessing a record/blob contains the database ID that was assigned to that content. The database ID is not persistent as deleting and reuploading the content will result in it being assigned a different database ID. This is especially relevant when restoring a database from a backup as the database IDs are not retained.

Follow the following steps if you wish to use PIDs in your databse

1. Depending on your CDCS version, there may be a database setting that the person administering your database needs to toggle to allow PIDs.
2. Controlling PID settings requires having sufficiently high usage privileges to assign XPATHs and adjust the settings.  Check to see if you have these permissions (can access Admin DashBoard and the PID Settings within).
3. The PIDs are assigned according to specific element values in the records.  Setting PID XPATHs on a template-specific basis tells the database which element of the records of the associated template specifies the PID.
4. Each record that you wish to have a PID assigned to it must then have an element value at the PID XPATH location.  The value should be the full URL for the PID of the form {host}/pid/rest/{provider}/{prefix}/{record}.

   a. host is the host URL for the database, i.e. the URL homepage.
   
   b. provider classifies where the record comes from.  The typical value is "local".

   c. prefix defines a subdomain within the database which can help categorize the hosted records and assist with uniqueness. Check the PID Settings for the list of allowed prefixes in the CDCS you are using, and contact the admins if you want to add a new one.

   d. record then is some record-specific ID that makes the full URL unique.

6. PIDs should be automatically assigned to any record if the auto_set_pid setting is turned on and if the record has a valid PID URL in the correct XPATH location.

### 8.1. Toggling auto_set_pid setting

**READ THIS IF YOU PLAN ON USING PID XPATHS!!!**

The auto_set_pid setting makes it easy for records to have PIDs simply by providing URLs in the appropriate template elements.  However, the CDCS may throw an error associated with assigning a URL for a record if you attempt to upload or update a record with a PID XPATH while the auto_set_pid setting is on.  To avoid this issue, the auto_set_pid setting should be toggled off during any record upload.

The simplest way to do this is when using the upload_record() and update_record() methods to add the parameter setting "auto_set_pid_off=True".  This will automatically turn the setting off before the upload and back on afterwards.  The downside to using this option is that there is some overhead as it requires two extra REST calls for every upload.

If you are performing batch uploads/updates of multiple records then it is preferable to either manually toggle auto_set_pid or use the auto_set_pid_off() context manager as shown below.

In [53]:
# Interacting with the auto_set_pid attribute will perform the appropriate REST calls.
print(curator.auto_set_pid)

curator.auto_set_pid = False
print(curator.auto_set_pid)

curator.auto_set_pid = True
print(curator.auto_set_pid)

True
False
True


In [54]:
# Or, use the context manager to ensure auto_set_pid is turned back on when done.

# Setting should initially be on
print(curator.auto_set_pid)

# Upload records with setting turned off
with curator.auto_set_pid_off():
    print(curator.auto_set_pid)

# Setting automatically turned back on after exiting the context block
print(curator.auto_set_pid)

True
False
True


### 8.2. Exploring existing PID XPATHs


The currently set PID XPATHs can be retrieved and explored with get_pid_xpaths() and get_pid_xpath().

Parameters

- __template__ (*str or pandas.Series, optional*) The template or template title to limit the search by.

In [4]:
curator.get_pid_xpaths()

,id,path,template
0,2,free-surface.URL,10
1,3,stacking-fault.URL,11
2,4,point-defect.URL,12
3,5,interatomic-potential.URL,21
4,6,crystal-prototype.URL,13
5,7,potential-LAMMPS.URL,14
6,8,calculation-diatom-scan.URL,16
7,9,calculation-isolated-atom.URL,17
8,10,potential-LAMMPS-KIM.URL,18
9,11,dislocation.URL,20


In [5]:
curator.get_pid_xpath('free_surface')

id                         2
path        free-surface.URL
template                  10
Name: 0, dtype: object

### 8.3. Uploading and updating PID XPATHS

PID XPATHs can be set and changed with upload_pid_xpath() and update_pid_xpath(), respectively.
    
Parameters
    
- __template__ (*str or pandas.Series*) The template or template title.
- __xpath__ (*str*) The xpath to use for the pid field for the template.

In [6]:
curator.upload_pid_xpath('Action', 'action.somekey')
curator.get_pid_xpath('Action')

id                      12
path        action.somekey
template                 7
Name: 11, dtype: object

In [7]:
curator.update_pid_xpath('Action', 'action.someotherkey')
curator.get_pid_xpath('Action')

id                           12
path        action.someotherkey
template                      7
Name: 11, dtype: object

### 8.4. Deleting PID XPATHS

A set PID XPATH can be deleted with delete_pid_xpath().
    
Parameters
    
- __template__ (*str or pandas.Series*) The template or template title.

In [8]:
curator.delete_pid_xpath('Action')
curator.get_pid_xpaths('Action')

,id,path,template
